In [17]:
HOME = "C:\\Users\\Pelin\\Documents\\RedFlagListStudy\\"

In [18]:
import os
os.chdir(HOME)

In [19]:
import streamlit as st

import pandas as pd
import numpy as np
import uuid  # To generate unique IDs

import re 


In [20]:
file_path = "RedFlagQuestions_Scores.xlsx"

# functions

In [21]:
def is_valid_email(email):
    """
    Validate the email format using a simple regex.
    """
    regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(regex, email) is not None

In [22]:
def load_data(file_path):
    cat = pd.read_excel(file_path, sheet_name="Categories")
    dt = pd.read_excel(file_path, sheet_name="RedFlags_newCat")
    filters = pd.read_excel(file_path, sheet_name="Filters")
    return cat,dt,filters

In [23]:
def toxic_score_sofar(file_name='user_responses.csv'):
    try:
        dt = pd.read_csv(file_name)
        if not dt.empty:
            return dt['Toxic Score'].mean()
        else:
            return 0  # Default value if the file is empty
    except FileNotFoundError:
        return 0  # Default value if the file does not exist

In [24]:
def ask_filter_questions(filters, language):
    """
    Ask the filter questions and collect integer responses.
    """
    st.subheader("Additional Questions / Ek Sorular", divider=True)
    responses = {}
    filter_violations = 0
    for index, row in filters.iterrows():
        question = row[f"Filter_Question_{language}"]  # Get the question in the selected language
        upper_limit = row["Upper_Limit"]  # Get the upper limit for the response
        response = st.number_input(
            f"{question}", 
            min_value=0, 
            max_value=5, 
            key=f"filter_{index}"
        )
        responses[row["Filter_Name"]] = response

        if response > upper_limit:
            filter_violations += 1

    return responses,filter_violations

In [25]:
def generate_survey(data, language):
    answers = {}
    tot_score = 0
    abs_tot_score = 0
    for index, row in data.iterrows():
        question = row[f'Question_{language}'].strip()  # Assuming 'Question' is the column header for questions
        st.markdown(f"**{index + 1}.** **{question}**")

        #if "visibility" not in st.session_state:
        #    st.session_state.visibility = "visible"
        #    st.session_state.disabled = False
        #    st.session_state.horizontal = False


        #col1, col2 = st.columns(2)
        #with col1:
        #    st.radio(
        #        "Applicable",
        #        ["applicable", "not applicable"],
        #        key="visibility",
        #        label_visibility=st.session_state.visibility,
        #        disabled=st.session_state.disabled,
        #        horizontal=st.session_state.horizontal,
        #    )

        # Handle different scoring types
        scoring_type = row['Scoring']
        if scoring_type == "Range(0-10)":
            answer = st.slider(f"Please select a score (0-10):", min_value=0, max_value=10, key=f"slider_{index}")
        elif scoring_type == "YES/NO":
            answer = st.radio(f"Please select:", options=["Yes", "No"], key=f"radio_{index}")
            answer = 7 if answer == "Yes" else 0  # Convert "Yes" to 5 and "No" to 0
        else:
            st.error(f"Unknown scoring type: {scoring_type}")
            answer = 0  # Default to 0 if scoring type is unknown

        answers[f"Q{row['ID']}"] = answer 
        st.divider()
        ## scoring
        weight = row['Weight']
        tot_score       = tot_score + weight*answer
        abs_tot_score   = abs_tot_score + weight*np.where(weight<0, 0,10) # negatif weigtli bir şeyde toksik olmak 0 puan almak demektir.

    toxic_score = 1.0*tot_score/abs_tot_score
    return answers,toxic_score

In [26]:
# Function to save user info and answers to a CSV file
def save_user_data(user_id, name, email,language, answers, toxic_score, filter_responses, filter_violations,file_name='user_responses.csv'):
    # Create a dictionary to store user data and answers
    user_data = {
        'User ID': user_id,
        'Name': name,
        'Email': email,
        'Language': language,
        'Toxic Score': toxic_score,
        **answers,  # Unpack the answers dictionary
        **filter_responses,  # Unpack the filter responses dictionary
        'Filter Violations': filter_violations  # Add filter violations
    }
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame([user_data])
    
    # Append the data to the CSV file (or create a new file if it doesn't exist)
    try:
        existing_data = pd.read_csv(file_name)
        updated_data = pd.concat([existing_data, df], ignore_index=True)
    except FileNotFoundError:
        updated_data = df
    
    # Save the updated data to the CSV file
    updated_data.to_csv(file_name, index=False)

In [27]:
def welcome(name, language):
    if language == "EN":
        st.markdown(f"Hello :blue[**{name}**]! :sunglasses:")
        st.markdown(f"It is so nice to see you here. This survey is designed to help you to see how :red[toxic] your boyfriend is :bomb:")
        st.markdown("Please feel free to answer all the questions so that we can analyze the results obtained from all girls and use them to better point the toxic guys.")
    
    elif language == "TR":
        st.markdown(f"Merhaba {name}!")
        st.markdown(f"Burada seni görmek çok güzel. Bu anket, erkek arkadaşının ne kadar :red[toksik] olduğunu görmene yardımcı olmak için tasarlandı :bomb:")
        st.markdown("Lütfen tüm soruları cevaplamaktan çekinme, böylece tüm kızlardan elde edilen sonuçları analiz edebilir ve toksik erkekleri daha iyi tespit edebiliriz.")
    

In [28]:
def generate_feedback(language):
    if language == "EN":
        st.write("Please rate your experience:")
    elif language == "TR":
        st.write("Lütfen deneyiminizi değerlendirin:")

    # Use st.feedback to collect user feedback
    selected = st.feedback("stars")

    # Handle the case where no feedback is selected
    if selected is None:
        if language == "EN":
            st.warning("Please provide a rating before submitting.")
        elif language == "TR":
            st.warning("Lütfen göndermeden önce bir değerlendirme yapın.")
        return None  # Return None if no feedback is selected

    # Map the selected feedback to a sentiment
    sentiment_mapping = ["one", "two", "three", "four", "five"]
    st.markdown(f"You selected {sentiment_mapping[selected]} star(s).")

    return selected + 1  # Return the rating (1-5)

def save_feedback(user_id, name, email,language, rating,file_name='user_feedback.csv'):
    user_data = {
        'User ID': user_id,
        'Name': name,
        'Email': email,
        'Language': language,
        'Rating': rating
    }

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame([user_data])
    
    # Append the data to the CSV file (or create a new file if it doesn't exist)
    try:
        existing_data = pd.read_csv(file_name)
        updated_data = pd.concat([existing_data, df], ignore_index=True)
    except FileNotFoundError:
        updated_data = df
    
    # Save the updated data to the CSV file
    updated_data.to_csv(file_name, index=False)


# dashboard

In [33]:
#language = 'TR'
#cat,dt,filters = load_data(file_path)
#data = dt.copy()
#answers = {}
#tot_score = 0
#abs_tot_score = 0
#for index, row in data.iterrows():
#    question = row[f'Question_{language}'].strip()  # Assuming 'Question' is the column header for questions
#    print(question)

In [ ]:
#if "user_details" not in st.session_state:
#    #st.session_state.user_details = {"name": None, "email": None, "language": None}
#    st.session_state.user_details = {"name": 'pelin_deneme', "email": 'pelin@deneme.com', "language": 'TR'}

In [29]:
def main():
    st.title("RedFlag List")

    # Initialize session state for user details and submission status
    if "user_details" not in st.session_state:
        st.session_state.user_details = {"name": 'pelin_deneme', "email": 'pelin@deneme.com', "language": 'TR'}
    if "filter_responses" not in st.session_state:
        st.session_state.filter_responses = None
        st.session_state.filter_violations = 0
    if "submitted" not in st.session_state:
        st.session_state.submitted = False  # Track if the survey has been submitted
    if "toxic_score" not in st.session_state:
        st.session_state.toxic_score = None  # Store the toxic score in session state
    if "avg_toxic" not in st.session_state:
        st.session_state.avg_toxic = None  # Store the average toxic score in session state

    # Ask the user for their preferred language
    if not st.session_state.user_details["language"]:
        st.write("Please select your preferred language / Lütfen tercih ettiğiniz dili seçin:")
        language = st.selectbox("Language / Dil", options=["EN", "TR"], key="language_select")
        if st.button("Submit / Gönder"):
            st.session_state.user_details["language"] = language
            st.rerun()
    # Ask the user for their name and email using a form
    elif not st.session_state.user_details["name"] or not st.session_state.user_details["email"]:
        language = st.session_state.user_details["language"]
        if language == "EN":
            st.write("Please enter your details:")
        elif language == "TR":
            st.write("Lütfen bilgilerinizi girin:")

        with st.form("user_details_form"):
            name = st.text_input("Name / İsim", key="name_input")
            email = st.text_input("Email / E-posta", key="email_input")
            if st.form_submit_button("Submit / Gönder"):
                if name and email:  # Ensure name and email are provided
                    if is_valid_email(email):  # Validate email format
                        st.session_state.user_details = {"name": name, "email": email, "language": language}
                        st.rerun()
                    else:
                        if language == "EN":
                            st.error("Please enter a valid email address (e.g., example@domain.com).")
                        elif language == "TR":
                            st.error("Lütfen geçerli bir e-posta adresi girin (örneğin, ornek@alanadi.com).")
                else:
                    if language == "EN":
                        st.error("Please enter both your name and email.")
                    elif language == "TR":
                        st.error("Lütfen hem adınızı hem de e-posta adresinizi girin.")
            
    else:
        name = st.session_state.user_details["name"]
        email = st.session_state.user_details["email"]
        language = st.session_state.user_details["language"]
        # Generate a unique ID for the user
        user_id = str(uuid.uuid4())  # Creates a unique identifier

        # Load the Excel file
        cat, dt, filters = load_data(file_path)

        # Generate the survey
        welcome(name, language)

        if st.session_state.filter_responses is None:
            st.write("Please answer the following filter questions:")
            filter_responses, filter_violations = ask_filter_questions(filters, language)
            if st.button("Submit Filter Responses / Filtre Cevaplarını Gönder"):
                st.session_state.filter_responses = filter_responses
                st.session_state.filter_violations = filter_violations
                st.rerun()  # Rerun the app to update the state
        else:
            if not st.session_state.submitted:
                if language == "EN":
                    st.subheader("Please answer the following questions :tulip:", divider=True)
                elif language == "TR":
                    st.subheader("Lütfen aşağıdaki soruları cevaplayın :tulip:", divider=True)
                answers, toxic_score = generate_survey(dt, language)

                # Save the user data and answers
                if st.button('Submit / Gönder'):
                    if name and email:  # Ensure name and email are provided
                        avg_toxic = toxic_score_sofar(file_name='user_responses.csv')

                        if st.session_state.filter_responses is not None:
                            save_user_data(user_id, name, email, language, answers, toxic_score, st.session_state.filter_responses, st.session_state.filter_violations)

                            # Store the toxic score and average toxic score in session state
                            st.session_state.toxic_score = toxic_score
                            st.session_state.avg_toxic = avg_toxic

                            # Mark the survey as submitted
                            st.session_state.submitted = True
                            st.rerun()  # Rerun the app to show the feedback component
                        else:
                            st.error("Please complete the filter questions before submitting.")
                    else:
                        if language == "EN":
                            st.error("Please enter your name and email before submitting.")
                        elif language == "TR":
                            st.error("Göndermeden önce lütfen adınızı ve e-posta adresinizi girin.")

            # Show feedback only after submission
            if st.session_state.submitted:
                toxic_score = st.session_state.toxic_score  # Retrieve the toxic score from session state
                avg_toxic = st.session_state.avg_toxic  # Retrieve the average toxic score from session state
                if language == "EN":
                    st.success(f"Thank you for completing the survey! Your boyfriend's toxic score is: **{toxic_score:.2f}**")
                    if st.session_state.filter_violations > 0:
                        st.warning("However, unfortunately he failed the filters. This should be a critical warning for you :( !")
                    else:
                        if toxic_score < avg_toxic:
                            st.warning("Your boyfriend seems to have lower toxicity compared to many guys. Good for him!")
                        else:
                            st.error("Your score indicates a high level of toxicity. Please take action to address this.")
                elif language == "TR":
                    st.success(f"Anketi tamamladığınız için teşekkürler! Erkek arkadaşınızın toksiklik puanı: **{toxic_score:.2f}**")
                    if st.session_state.filter_violations > 0:
                        st.warning("Ama, ne yazık ki filtrelerde sınıfta kaldı. Bu senin için ciddi bir uyarı anlamına gelmeli :( !")
                    else:
                        if toxic_score < avg_toxic:
                            st.warning("Erkek arkadaşınız ortalamaya göre daha düşük toksiklik seviyesinde. Bu onun için iyi!")
                        else:
                            st.error("Skorunuz yüksek bir toksiklik seviyesini gösteriyor. Lütfen bu konuda harekete geçin.")

                # Collect feedback after submission
                rating = generate_feedback(language)
                if rating is not None:  # Only save feedback if a rating is provided
                    save_feedback(user_id, name, email, language, rating)
                    st.session_state.submitted = False  # Reset submission state for a new survey
                    st.session_state.toxic_score = None  # Reset toxic score
                    st.session_state.avg_toxic = None  # Reset average toxic score
                    st.rerun()  # Rerun the app to reset the UI
                else:
                    st.warning("Please provide feedback before proceeding.")


In [30]:
# Run the application
if __name__ == "__main__":
    main()

2025-02-09 00:03:53.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.620 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.624 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 00:03:53.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

KeyError: 'language'